#### SWETHA JENIFER S
#### 225229142

### PDL Lab16. Design of LSTM and GRU RNN for classification of IMDB reviews

In [4]:
import pandas as pd 
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRU 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model 

In [19]:
data = pd.read_csv('imdb.csv', encoding='latin1')
data.head()

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0


In [28]:
import nltk 
nltk.download('stopwords')
english_stops = set(stopwords.words('english'))
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sweth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
english_sto = set(stopwords.words('english'))

In [44]:
def load_dataset():
    df = pd.read_csv('imdb.csv', encoding='latin1')
    x_data = df['text']
    y_data = df['label']
    x_data = x_data.replace({'[’A-Za-z]' : ' '}, regex = True)
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_sto]) 
    x_data = x_data.apply(lambda review: [w.lower() for w in review])
    y_data = y_data.replace('positive', 1) 
    y_data = y_data.replace('negative', 0)
    return x_data, y_data 
x_data, y_data = load_dataset() 
print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)


Reviews
0     [', 14, -, ', ., ,, ,, ', ., ", 2", -, ", ", ....
1     [..., ..., ', ..., ..., ..., ., ?, ., ., $5.99...
2                                       [., ., ,, ., .]
3     [-, é, ., ,, ,, ., ', ., ,, (, ,, ,, ).<, /><,...
4     [-, ,, ,, ., ,, ', (, ), .<, /><, />, ', ,, ',...
                            ...                        
81    [., ', ..., ., ', -, -, -, ., (, ).<, /><, />,...
82    [,, 6, .<, /><, />, ', 2, ,, 11, 2001., ,, (, ...
83    [., -, -, ., ,, ,, ., ., ., -, -, ,, ', ,, ,, ...
84    [., ,, ,, ', ., ,, ", ", ., ., ', ., ,, ', /, ...
85    [,, ', ", /, ", ,, ., (, ', ), ", "., (, ), .,...
Name: text, Length: 86, dtype: object 

Sentiment
0     0
1     0
2     0
3     0
4     0
     ..
81    0
82    1
83    1
84    0
85    0
Name: label, Length: 86, dtype: int64


In [45]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.4)
print('Train Set') 
print(x_train, '\n') 
print(x_test, '\n') 
print('Test Set') 
print(y_train, '\n') 
print(y_test)

Train Set
1     [..., ..., ', ..., ..., ..., ., ?, ., ., $5.99...
14    [', ', (, 10, ), ., ', ', ,, ', .<, /><, />', ...
6     [(, é), ,, ., ,, -, ., (, é, ),, ,, ., ,, ', (...
80    [,, ., ', -, .<, /><, />, ', ., ,, .<, /><, />...
46    [90', ', '., ., ., ', ., ., ., ., ,, ,, ', ., ...
34    [..., ..., ', ., ., -, ', ..., ..., ', ., ', -...
32    [., ,, ., ,, ,, .<, /><, />, ,, ,, .<, /><, />...
12    [,, <, /><, />, ., 9, ,, <, /><, />, ,, 4, <, ...
33    [-, ,, ", ", -, ", -, ", ., ', ,, ,, -, ', ,, ...
52    [,, --, --, ., ,, ., ,, --, ', --, ,, 11, 09, ...
78    [-8,, (, ), ., 2, (, ), .<, /><, />, -, ?, ,, ...
54                                [,, ', ., 4, ., ,, .]
45                                         [., ., ., .]
68    [1937, ,, ,, ,, 20, ., ", ", (", ", ", "), ., ...
10                             [', ,, ., ,, ., ,, ,, .]
41    [:, <, /><, />1), (, ), ,, ., ,, (, ", "), ,, ...
8     [., ,, ., ,, ..., ,, ., ., ', ..., ., ., ,, (,...
7     [", ", ", ", ., ., ., ', ?, \, .

In [46]:
def get_max_length(): 
    review_length = []
    for review in x_train: 
        review_length.append(len(review))
    return int(np.ceil(np.mean(review_length)))

In [47]:
token   =   Tokenizer(lower=False)
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test) 
max_length = get_max_length()
x_train=pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post') 
x_test=pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1

print(' Encoded X Train\n', x_train, '\n') 
print(' Encoded X Test\n', x_test, '\n') 
print('Maximum review length: ', max_length)


 Encoded X Train
 [[ 10  10   3 ...   0   0   0]
 [  3   3   9 ...   2   1   8]
 [  9  58   1 ...  23   4  37]
 ...
 [  2   1   1 ...   5   2   2]
 [  1  11   3 ...   3 144   3]
 [  1   1   8 ...   0   0   0]] 

 Encoded X Test
 [[ 2  2  1 ...  0  0  0]
 [ 3  3  5 ...  3  2  1]
 [ 1  1  3 ...  2  3 11]
 ...
 [11 11  6 ...  0  0  0]
 [ 1  2  2 ...  0  0  0]
 [ 1  1  1 ...  8  6  7]] 

Maximum review length:  51


In [51]:
EMBED_DIM = 32
LSTM_OUT = 64

# Assuming x_data is a list of lists, where each inner list contains words
sentences = [' '.join(sentence) for sentence in x_data]

# Calculate the maximum sequence length
max_sequence_length = max(len(sentence.split()) for sentence in sentences)

# Set your_input_length to the maximum sequence length
your_input_length = max_sequence_length

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length=your_input_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 156, 32)           4640      
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 31585 (123.38 KB)
Trainable params: 31585 (123.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [60]:
modell = Sequential()
modell.add(Embedding(total_words, 32, input_length=max_length)) 
modell.add(LSTM(32))
modell.add(Dense(32, activation='relu')) 
modell.add(Dense(1, activation='sigmoid'))
modell.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 
modell.fit(x_train, y_train, batch_size = 128, epochs = 10, callbacks=[checkpoint]) 
modell.evaluate(x_test, y_test)
print(modell.summary())


Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.6951 - accuracy: 0.4510
Epoch 1: accuracy improved from -inf to 0.45098, saving model to models\LSTM.h5
1/1 [==============================] - 3s 3s/step - loss: 0.6951 - accuracy: 0.4510
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.6937 - accuracy: 0.5294
Epoch 2: accuracy improved from 0.45098 to 0.52941, saving model to models\LSTM.h5


C:\Users\sweth\Downloads\nlp\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 110ms/step - loss: 0.6937 - accuracy: 0.5294
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.6925 - accuracy: 0.5294
Epoch 3: accuracy did not improve from 0.52941
1/1 [==============================] - 0s 40ms/step - loss: 0.6925 - accuracy: 0.5294
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.6918 - accuracy: 0.6471
Epoch 4: accuracy improved from 0.52941 to 0.64706, saving model to models\LSTM.h5
1/1 [==============================] - 0s 118ms/step - loss: 0.6918 - accuracy: 0.6471
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.6910 - accuracy: 0.6275
Epoch 5: accuracy did not improve from 0.64706
1/1 [==============================] - 0s 41ms/step - loss: 0.6910 - accuracy: 0.6275
Epoch 6/10
1/1 [==============================] - ETA: 0s - loss: 0.6901 - accuracy: 0.6275
Epoch 6: accuracy did not improve from 0.64706
1/1 [==============================] - 0s 47ms/step - loss: 0.

In [63]:
model2 = Sequential()
model2.add(Embedding(total_words, 32, input_length=max_length)) 
model2.add(Bidirectional(LSTM(32)))
model2.add(Dense(32, activation='relu')) 
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 
model2.fit(x_train, y_train, batch_size = 128, epochs = 10)
model2.evaluate(x_test, y_test) 
print(model2.summary())


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6931 - accuracy: 0.5098
Epoch 2/10
1/1 [==============================] - 0s 61ms/step - loss: 0.6911 - accuracy: 0.5882
Epoch 3/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6897 - accuracy: 0.5686
Epoch 4/10
1/1 [==============================] - 0s 47ms/step - loss: 0.6885 - accuracy: 0.5882
Epoch 5/10
1/1 [==============================] - 0s 57ms/step - loss: 0.6872 - accuracy: 0.6275
Epoch 6/10
1/1 [==============================] - 0s 48ms/step - loss: 0.6857 - accuracy: 0.6471
Epoch 7/10
1/1 [==============================] - 0s 48ms/step - loss: 0.6841 - accuracy: 0.6471
Epoch 8/10
1/1 [==============================] - 0s 47ms/step - loss: 0.6822 - accuracy: 0.6275
Epoch 9/10
1/1 [==============================] - 0s 51ms/step - loss: 0.6799 - accuracy: 0.6275
Epoch 10/10
2/2 [==============================] - 1s 13ms/step - loss: 0.7042 - accuracy: 0.4857
Model: "sequential_3"
_________

In [64]:
model2.evaluate(x_test, y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.7042 - accuracy: 0.4857


[0.7042022347450256, 0.48571428656578064]

In [74]:
from keras.preprocessing.text import Tokenizer

t = Tokenizer()
# De fin i ng 4 doc men t L i sts
fit_text = ['Machine Learning Knowledge', 'Machine Learning',
'Deep Learning',
'Artificial Intelligence'] 
t.fit_on_texts(fit_text)


In [75]:
t